In [138]:
PATH_TO_YOLO_FOLDER = '/Users/dominikocsofszki/Desktop/drive_fake/MyDrive/yolov5'
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import yaml


#filters out duplicates by center_widht -> problem float comparing > rounding up to 2 decimal
# works but depends on the image size and ammount of labels > finding solution in detectlabelme
#-not used yet?
def filter_duplicates_given_names(df, round_nr):
    filter = df.round(round_nr).duplicated(subset=['cw'], keep='first')
    return df[~filter]

#handing over the labels for filtering out all the other label_center which are in that area
#ToDo delete filtered out items
#-not used yet?
def return_filter_cw_minmax_ch_minmax(tub,df:pd.DataFrame):
    (cw_min, cw_max, hw_min, hw_max) = tub
    # print(f'{cw_min}, {cw_max}, {hw_min}, {hw_max}')
    filter_boundries = (cw_min <= df['cw']) & (df['cw'] < cw_max) & (hw_min < df['ch']) & (df['ch'] < hw_max)
    return filter_boundries


#filters all elements that lay in the label boundry of one label.
#row => is one label entry.example: 25 0.189063 0.130469 0.046875 0.0578125 0.287464
def final_return_filter_duplicates(row,df:pd.DataFrame):
    # (cw_min, cw_max, hw_min, hw_max) = row["cw"] - (row['w'] / 2), row["cw"] + (row['w'] / 2), row["ch"] - (
    #             row['h'] / 2), row["ch"] + (row['h'] / 2),
    cw_min = row["cw"] - (row['w'] / 2)
    cw_max = row["cw"] + (row['w'] / 2)
    hw_min = row["ch"] - (row['h'] / 2)
    hw_max = row["ch"] + (row['h'] / 2)
    filter_boundries = (cw_min <= df['cw']) & (df['cw'] < cw_max) & (hw_min < df['ch']) & (df['ch'] < hw_max)
    return filter_boundries


#for birectional keys. Return for the class_nr the value
def get_key(my_dict, val):  #better way for sure possible
    for key, value in my_dict.items():
        if val == value:  #string compare no equals???
            return key


#reads the yaml file, adds the path:... to the yaml files and saves it
#NOT 100% SURE IF WRITING THE PATH WORKS YET, TO BE TESTED, BUT NOT NEEDED YET
#returns the yaml file for next function dict
def change_and_return_yaml_dict(path_to_yaml_folder):
    label_folder = Path(path_to_yaml_folder)
    fname = label_folder / 'data.yaml'
    with open(fname) as f:
        yaml_load = yaml.safe_load(f)
    yaml_load["path"] = str(label_folder.resolve())
    with open(fname, "w") as f:
        yaml.dump(yaml_load, f, default_flow_style=False)
        return yaml_load


#reads the yaml file and returns a dict for all the data.yaml entries and
# returns the dict_class for translating class_nr to class_name
def dict_classes_and_yaml(data_yaml_source):
    dict_yaml = change_and_return_yaml_dict(data_yaml_source)
    dict_classes = dict(enumerate(map(str, dict_yaml.get('names'))))
    return dict_classes, dict_yaml

#reads the csv and add the headers
#cw = center width, ch = center height, w = width of label, h = height of label, p = accuracy of label
def read_csv_new_header(csv_txt_path) :
    # df = pd.read_csv(csv_txt_path,header=None,names=('class_nr','ch','cw','h','w','p'))
    df = pd.read_csv(csv_txt_path,names=['class_nr','cw','ch','w','h','p'],delimiter=' ')
    return df

##WORKING PLEASE NO CHAGE!!
#checks all labels in one image
#here you can also change the colour of the first x labels, since they have the highest accuracy,
#You can also add a list with items that have been change, to make changes visible
def check_label_all(df:pd.DataFrame, img, above_center=-30, first_x_labels_diff_colour=50,
                    list_index_changed_before=None):
    if list_index_changed_before is None:
        list_index_changed_before = []      #showing changes if any
    plt.rcParams['figure.dpi'] = 250    #<change if img to small#increase size to be better able to read images on screen
    plot, ax = plt.subplots()
    w = img.shape[0]
    h = img.shape[1]
    col_changing = 'green'        # picking coulour to show

    for i in range(df.shape[0]):
        df_entry = df.iloc[i]
        hmp = df_entry[1] * h       #calculate borders in pixel size, before was i percentage of img
        wmp = df_entry[2] * w
        # print(df_entry)
        # col_changing = 'red'        # picking coulour to show
        # if i <= first_x_labels_diff_colour: col_changing = 'pink' ##Not so bright entries for high accuracy
        if i in list_index_changed_before: col_changing = 'black'  ##<<----Will not work, after first all will change to green ##Changed labels
        if i == 50 :
            # above_center = above_center+10
            col_changing = 'red'
        if i == 100 :
            # above_center = above_center+10
            col_changing = 'black'
        ax.plot(hmp, wmp + above_center, marker='$' + str(i) + ',' + '$', color=col_changing, ms=8)
    # plt.title("test")
    ax.plot()
    # ax.imshow(plt.imread(path_to_image))
    ax.imshow(img)
    ax.savefig(('TEST_cp'), bbox_inches='tight')

    ax.plot()
##until

##To only show the line with the selected row, row(indx)
def final_filter_out_one_line(df:pd.DataFrame,indx:int) :
    filter = return_filter_cw_minmax_ch_minmax((0,1,df.iloc[indx]['cw']-df.iloc[indx]['w']/2,df.iloc[indx]['cw']+df.iloc[indx]['w']/2),df=df)
    return filter

#dict to find all paths related to Yolov5 folder
##If you want to change sth, better add a new dict with new path instead of changing any old ones!
def path_to_yolo_return_dict_paths(PATH_TO_YOLO_FOLDER):
    yolo = Path(PATH_TO_YOLO_FOLDER).absolute()
    print(f'yolo:{yolo}')
    labelme = yolo.parent / 'labelme'
    detect_labelme = labelme / 'detect_labelme.py'
    runs = labelme / 'runs'
    override_no_inc = runs / 'detect' / 'override_no_inc'
    copy = labelme / 'copy'
    weight = labelme / 'weight'
    weight_best = labelme / 'weight' / 'best.pt'
    detect = yolo / 'detect.py'
    detect_folder = runs / 'detect'
    #labelme.py
    labelme_py = yolo/'detect_labelme.py'
    train = yolo / 'train.py'
    labels = override_no_inc / 'labels'
    source = override_no_inc / 'source'
    yaml = weight / 'data.yaml'
    original_source = labelme / 'source'
    labelme_source = labelme / 'source'
    screenshots = labelme/'source'

    dict_paths = {
        'labelme_py':labelme_py,
        'labelme_source': labelme_source,
        'detect_folder': detect_folder,
        'original_source': original_source,
        'yaml': yaml,
        'original': source,
        'labels': labels,
        'labelled_pic': override_no_inc,
        'override': override_no_inc,
        'train': train,
        'detect': detect,
        'yolo': yolo,
        'labelme': labelme,
        'source': source,
        'detect_labelme': detect_labelme,
        'runs': runs,
        'copy': copy,
        'weight': weight,
        'weight_best': weight_best,
        'screenshots': screenshots
    }
    for x in dict_paths:
        asd=''
        # print(f'{x}: {dict_paths[x]}')
    return dict_paths

dict_path = path_to_yolo_return_dict_paths(PATH_TO_YOLO_FOLDER) #produces the dict for paths


# x = dict_path['labels'].iterdir()       # gives us a list of all labels in the folder

# entries = [entry for entry in dict_path['labels'].iterdir()]    #makes the label list iterable

def return_path_entries_folder(path_to_img_folder:Path) :
    entries = [entry for entry in path_to_img_folder.iterdir()] ##need to ignore other files!?!?<<<<<<<<
    entries.sort()
    return entries

def full_run_labelme(above_center:int=35) :
    dict_classes, _ = dict_classes_and_yaml(dict_path['weight']) # path to data.yaml, top folder
    # entries_images = return_path_entries_folder(dict_path['source'])
    # entries_images = return_path_entries_folder(dict_path['screenshots'])
    entries_images = return_path_entries_folder(dict_path['override'])
    # entries_labels = return_path_entries_folder(dict_path['labels'])
    entries_labels = return_path_entries_folder(dict_path['labels'])
    path_to_labels = dict_path['labels']
    for img_path in entries_images :
        if not img_path.is_file() : continue        ##Ignore items that are not files
        # labels_path = entries_labels/img_path.stem/'.txt'
        labels_path = (path_to_labels/(img_path.stem+'.txt'))
        ##
        if not labels_path in entries_labels :      ##Happens if nothing was detected
            print(f'Missing Label: {img_path.stem}')    ##Could also put it into a file later
            continue
        ##
        df = read_csv_new_header(labels_path)       ##Reads the .txt file
        # dict_classes[]

        df['dict'] = df['class_nr'].map(dict_classes) ## produces a new Colum with the dict to the class_nr

        # print(df)
        img = plt.imread(img_path)  ##reads the image
        #here i wanted to filter out all duplicates with different function,

        for i in range(df.shape[0]):        ##For filtering out duplicates >>>Not used yet
            filter_find_duplicates_one_entry = final_return_filter_duplicates(df.iloc[i],df=df)
            #here i wanted to filter out all duplicates with different function,

            #needs to be finished, for now we use the other filter
        # df_noduplicate = filter_duplicates_given_names(df, 3) ## Filters out duplicates, rounds on float,3
        df_sort_for_cw = df.sort_values('p')

        df_sort_or_not = df
        # check_label_all(df=df_sort_for_cw,img=img,above_center = - above_center, first_x_labels_diff_colour=200)##calc above
        check_label_all(df=df_sort_or_not,img=img,above_center = - above_center, first_x_labels_diff_colour=200)##calc above
        plt.title(str(img_path.name))  ##add it here <<<
        plt.savefig((img_path.resolve()+'_cp'), bbox_inches='tight')

        df_sort_or_not.to_csv(labels_path.with_suffix('.csv'),sep=',')
        # df_sort_for_cw.to_csv(labels_path.with_suffix('.csv'),sep=',')
        # df.to_csv()
        # input()       ##<<delete me
        # break

yolo:/Users/dominikocsofszki/Desktop/drive_fake/MyDrive/yolov5


In [146]:
def read_csv_with_dict(csv_txt_path) :
    # df = pd.read_csv(csv_txt_path,header=None,names=('class_nr','ch','cw','h','w','p'))
    # df = pd.read_csv(csv_txt_path,names=['class_nr','cw','ch','w','h','p','dict'],delimiter=',',header = None)
    # df = pd.read_csv(csv_txt_path,names=['class_nr','cw','ch','w','h','p','dict'],delimiter=',',header=None)
    df = pd.read_csv(csv_txt_path,delimiter=',')
    # df = pd.read_csv(csv_txt_path,delimiter=' ',header=None)
    return df
def return_dict_class_name_to_nr(dict_classes):
    return dict([reversed(i) for i in dict_classes.items()])

    # revd=dict([reversed(i) for i in dict_classes.items()])
    # print(revd)
    # dict_classes.update(revd)
    # return revd


def read_changed_label_csv_into_label_txt(csv_path) :
    dict_classes, _ = dict_classes_and_yaml(dict_path['weight']) # path to data.yaml, top folder
    dict_class_reverse = return_dict_class_name_to_nr(dict_classes)
    df = read_csv_with_dict(csv_path)
    #1.read .csv into       <<<<<<<<<<<<<<<<<<<<<<<<
    #2.
    ######<<<<<
    print(df)
    #2.
    # df.drop(labels='class_nr')
    df['class_nr'] = df['dict'].map(dict_class_reverse) ##Check if it works
    # 2. translate from dict to class_nr
    ##
    #ToDo drop
    #3.Drop df['dict'}
    df.to_csv(Path(csv_path).with_suffix('.txt'),sep=' ')
    #4. save/override the old name.txt file <<<<<<<<<<<<<<<<<<<



read_changed_label_csv_into_label_txt('/Users/dominikocsofszki/Desktop/drive_fake/MyDrive/labelme/runs/detect_6/labels/labme 2022-12-04 at 15.50.59.csv')


     Unnamed: 0  class_nr       cw       ch       w       h         p dict
0             0        60  0.04000  0.37125  0.0800  0.0375  0.264782    u
1             1        44  0.50500  0.94375  0.0650  0.0475  0.294922    e
2             2        57  0.69125  0.81125  0.0475  0.0425  0.299097    r
3             3        40  0.95375  0.24375  0.0725  0.0475  0.315310    c
4             4        58  0.32875  0.49625  0.0375  0.0625  0.341043    s
..          ...       ...      ...      ...     ...     ...       ...  ...
148         148        58  0.25250  0.81000  0.0300  0.0450  0.910084    s
149         149        58  0.70750  0.81000  0.0300  0.0450  0.910546    s
150         150        44  0.64125  0.81000  0.0375  0.0450  0.912641    e
151         151        44  0.52750  0.68125  0.0350  0.0425  0.914343    e
152         152        44  0.07500  0.94125  0.0350  0.0425  0.915322    e

[153 rows x 8 columns]


KeyError: "['class_nr'] not found in axis"

In [144]:
count_copies = 37

In [145]:


##Labels screenshots, upload in folder override_no_inc
DETECT_LABELME_PATH = '/Users/dominikocsofszki/Desktop/drive_fake/MyDrive/yolov5/detect_labelme.py'
# DETECT_LABELME_PATH = str(dict_path['labelme_py'])+'.py'
# print(DETECT_LABELME_PATH)

# !python {DETECT_LABELME_PATH} --device 'cpu' --max-det 100
# !python {DETECT_LABELME_PATH} --device 'cpu' --classes 0 1 2 3
!python {DETECT_LABELME_PATH} --device 'cpu' --agnostic-nms             #<<<148 items
# !python {DETECT_LABELME_PATH} --device 'cpu'                                ##178 items!!!!

# full_run_labelme(above_center = 35)

# !python {DETECT_LABELME_PATH} --device 'mps'

##runs over these images shows, cleans up duplicates, shows you the new labels
source = dict_path['override']
dest = source.parent
count_copies = count_copies + 1
dest_new = str(dest) +'_'+str(count_copies)
# print(source,dest)
full_run_labelme(above_center = -20)

# input('Check your new csv and change, press enter to load csv into .txt')
# ##Needs to be set before: count_copies = 0
# !mv {source} {dest_new}   #Moves the folder into labelme/runs -> deletes the old folder
# !cp -R {source} {dest_new}    ##Copies the folder into labelme/runs <<<<<<<<<<with this adds more and more to the text file!

##saves the new labels as csv
#->open csv and correct labels in there, index =>(should)=> equal number on screen

#read and transform csv into name.txt
# read_changed_label_csv_into_label_txt()

input('Check your new csv and change, press enter to load csv into .txt')
##Needs to be set before: count_copies = 0
!mv {source} {dest_new}   #Moves the folder into labelme/runs -> deletes the old folder
print(f'moved {source} to {dest_new}')

/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefINS2_6SymIntEEENS2_8optionalINS2_10ScalarTypeEEENS6_INS2_6LayoutEEENS6_INS2_6DeviceEEENS6_IbEENS6_INS2_12MemoryFormatEEE
  Referenced from: <676A2B44-47B6-3691-A0F0-5CD286E230FA> /opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <BB02660F-1D5B-3388-B48B-486877D726F6> /opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")
yolo:/Users/dominikocsofszki/PycharmProjects/checklabel/../../Desktop/drive_fake/MyDrive/yolov5
labelme_source: /Users/dominikocsofszki/PycharmProjects/checklabel/../../D

In [ ]:
input('Check your new csv and change, press enter to load csv into .txt')
##Needs to be set before: count_copies = 0
!mv {source} {dest_new}   #Moves the folder into labelme/runs -> deletes the old folder